In [ ]:
# Install PyTorch Geometric (versi yang stabil di Colab)
!pip install torch==2.1.0 torchvision torchaudio
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-geometric




Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.0 MB/s eta 0:00:00


In [ ]:
!pip install numpy==1.26.4 --force-reinstall



  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import gdown

# Ganti ini dengan ID file dari link berbagi Google Drive
file_id = '1eRA8ZCgpODwOpAm3nsgg4_wzUCjcy9Vm'
output = 'graph_dataset_fresh_1.csv'  # Ganti sesuai nama file yang diinginkan
gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1eRA8ZCgpODwOpAm3nsgg4_wzUCjcy9Vm
From (redirected): https://drive.google.com/uc?id=1eRA8ZCgpODwOpAm3nsgg4_wzUCjcy9Vm&confirm=t&uuid=d16ffc14-fddc-40cd-aea0-a9ed41b8de5a
To: /workspace/graph_dataset_fresh_1.csv
100%|██████████| 7.90G/7.90G [03:18<00:00, 39.7MB/s]


'graph_dataset_fresh_1.csv'

In [ ]:
!pip install pandas numpy torch
!pip install scikit-learn



In [ ]:
import pandas as pd
import numpy as np
import torch
import ast
from sklearn.decomposition import PCA
from torch_geometric.data import Data
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- 1. BACA CSV ---
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv'

In [ ]:
# Lihat tipe data setiap elemen
print(type(df.loc[24000, "Node_Features"]))  # List atau string?

# Kalau string, kemungkinan parsing JSON yang bermasalah


<class 'str'>


In [ ]:
import pandas as pd
import numpy as np
import json

all_features = np.array([json.loads(s) for s in df["Node_Features"]])



In [ ]:
from sklearn.preprocessing import StandardScaler
# Ambil 9 fitur pertama (color moments)
color_moments_all = all_features[:, :, :9]  # shape: [N, V, 9]

# Flatten ke 2D untuk fit scaler
color_moments_flat = color_moments_all.reshape(-1, 9)

# Normalisasi
scaler = StandardScaler()
color_moments_flat_scaled = scaler.fit_transform(color_moments_flat)

# Kembalikan ke bentuk semula
color_moments_scaled = color_moments_flat_scaled.reshape(all_features.shape[0], all_features.shape[1], 9)

# Ambil 3 fitur terakhir (x, y, rasio_aspek) tanpa diubah
spatial_features = all_features[:, :, 9:]

# Gabungkan hasil normalisasi + fitur spatial
normalized_all_features = np.concatenate([color_moments_scaled, spatial_features], axis=-1)



In [ ]:
# Update kembali ke variabel utama
all_features = normalized_all_features

In [ ]:
all_features = color_moments_scaled


In [ ]:
# Simpan kembali ke kolom dataframe
df["Node_Features"] = [json.dumps(f.tolist()) for f in normalized_all_features]

In [ ]:
import ast
import numpy as np

batch_size = 100
total = len(df)

for start in range(0, total, batch_size):
    end = min(start + batch_size, total)
    print(f"🔄 Processing batch {start} to {end - 1}...")

    # Ambil subset batch
    subset = df["Node_Features"].iloc[start:end]

    # Parse list of string → list of list
    batch_features = np.array([ast.literal_eval(s) for s in subset])

    # Simpan hasil batch
    np.save(f"/workspace/batch_features_{start}_{end-1}.npy", batch_features)

    print(f"✅ Saved batch {start}–{end - 1} | shape: {batch_features.shape}")


In [ ]:
# --- 2. Parse Node Features ---
all_features = np.array([ast.literal_eval(s) for s in df["Node_Features"]])  # shape: [num_graph, num_node, 2828]
num_graphs, num_nodes, feat_dim = all_features.shape


In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Load data fitur node (pastikan array 3D [N, node, feat_dim]) ---
all_features = np.load("/workspace/all_node_features.npy")  # atau path kamu
num_graphs, num_nodes, feat_dim = all_features.shape

# --- Flatten semua node dari semua graph ---
flat = all_features.reshape(-1, feat_dim)  # [N_graph * N_node, feat_dim]

# --- Fit PCA tanpa n_components untuk ambil seluruh varian ---
pca = PCA()
pca.fit(flat)

explained_var = pca.explained_variance_ratio_
cumulative_var = np.cumsum(explained_var)

# --- Buat DataFrame untuk analisis ---
dimensi = np.arange(1, len(explained_var)+1)
df_pca = pd.DataFrame({
    "n_components": dimensi,
    "explained_variance_ratio": explained_var,
    "cumulative_variance": cumulative_var
})

# --- Simpan ke CSV ---
df_pca.to_csv("/workspace/analisis_dimensi_pca.csv", index=False)
print("📁 Disimpan ke: /workspace/analisis_dimensi_pca.csv")

# --- Cetak beberapa baris penting ---
print(df_pca[df_pca["n_components"].isin([50, 100, 128, 256, 512])])

# --- Plot scree (elbow) plot ---
plt.figure(figsize=(10, 5))
plt.plot(dimensi, cumulative_var, marker='o')
plt.axhline(y=0.90, color='r', linestyle='--', label='90%')
plt.axhline(y=0.95, color='g', linestyle='--', label='95%')
plt.xlabel("Jumlah Komponen PCA")
plt.ylabel("Varians Kumulatif")
plt.title("Scree Plot (Cumulative Explained Variance)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/workspace/scree_plot_pca.png")
plt.show()


In [ ]:
# --- 3. PCA ---
flat = all_features.reshape(-1, feat_dim)  # shape: [num_graphs * num_nodes, 2828]
pca = PCA(n_components=512)
flat_reduced = pca.fit_transform(flat)     # shape: [num_graphs * num_nodes, 512]
reduced = flat_reduced.reshape(num_graphs, num_nodes, -1)  # [N, 64, 512]

In [ ]:
# --- 4. Build Graph DataList ---
data_list = []

for i in range(len(df)):
    row = df.iloc[i]
    x = torch.tensor(reduced[i], dtype=torch.float)

    edge_index = torch.tensor(ast.literal_eval(row["Edge_Index"]), dtype=torch.long).t().contiguous()
    y = torch.tensor(0 if row["Category"] == "tidak_estetik" else 1, dtype=torch.long)

    data = Data(x=x, edge_index=edge_index, y=y)
    data.file_name = row["Filename"]
    data_list.append(data)

# --- 5. Save to .pt ---
torch.save(data_list, "/content/drive/MyDrive/Hasil_Ekstraksi/color/processed_graphs_pca.pt")

# --- 6. Simpan model PCA juga jika perlu ---
joblib.dump(pca, "/workspac/pca_model_512.joblib")

print(f"Sukses simpan {len(data_list)} graf ke .pt!")

NameError: name 'reduced' is not defined

In [ ]:
# --- 3. Skip PCA ---
reduced = all_features  # langsung gunakan fitur asli

# --- 4. Build Graph DataList ---
data_list = []

for i in range(len(df)):
    row = df.iloc[i]
    x = torch.tensor(reduced[i], dtype=torch.float)

    edge_index = torch.tensor(ast.literal_eval(row["Edge_Index"]), dtype=torch.long).t().contiguous()
    y = torch.tensor(0 if row["Category"] == "tidak_estetik" else 1, dtype=torch.long)

    data = Data(x=x, edge_index=edge_index, y=y)
    data.file_name = row["Filename"]
    data_list.append(data)

# --- 5. Save to .pt ---
torch.save(data_list, "/content/drive/MyDrive/Hasil_Ekstraksi/color_only/processed_graphs_color__only_normal.pt")
print(f"Sukses simpan {len(data_list)} graf ke .pt tanpa PCA!")


Sukses simpan 24010 graf ke .pt tanpa PCA!


In [ ]:
# Path file input & output
input_path = "/content/drive/MyDrive/Hasil_Ekstraksi/color_only/processed_graphs_color__only_normal.pt"
output_path = input_path  # bisa diubah kalau mau simpan ke file berbeda

# Load data
data_list = torch.load(input_path, weights_only=False)

print("Jumlah grafik:", len(data_list))
print("Tipe objek pertama:", type(data_list[0]))
print("Isi atribut:", data_list[0])

Jumlah grafik: 24010
Tipe objek pertama: <class 'torch_geometric.data.data.Data'>
Isi atribut: Data(x=[9, 9], edge_index=[12, 2], y=0, file_name='13089.jpg')


In [ ]:
print(data_list[0].x)


tensor([[ 0.8040,  0.7817, -1.5204,  1.0291,  0.5050, -1.7911, -1.4900, -0.7901,
          2.9241],
        [ 1.0133,  0.0776, -1.3816,  1.2940, -0.0694, -1.2891, -1.5048, -1.2298,
          2.3220],
        [ 0.9726, -0.2328, -1.4947,  1.1972, -0.3643, -1.3903, -1.5647, -1.3605,
          3.1253],
        [ 1.0921,  0.4907, -1.5202,  1.5668,  0.4114, -1.8303, -1.3100, -0.0435,
          1.1986],
        [ 1.2095,  0.7174, -1.0471,  1.5153,  0.7658, -1.2273, -1.4221, -0.5956,
          1.2887],
        [ 0.7418,  1.2133, -0.8276,  0.9671,  1.1519, -1.0776, -1.5307, -0.4974,
          2.0701],
        [ 0.6677,  0.2376, -1.3634,  0.8047,  0.0501, -1.6757, -1.5432, -1.0952,
          4.1057],
        [ 1.0723,  0.2839, -1.5212,  1.1827,  0.1987, -1.6204, -1.5958, -1.6205,
          2.1676],
        [ 1.2371, -0.2766, -1.8026,  1.3901, -0.1820, -1.6631, -1.5020, -1.1078,
          2.2989]])
